<a href="https://colab.research.google.com/github/blid11/Group9/blob/Brett/Copy_of_G9_PartC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Changes made in this Colab notebook can be saved with Control/Command S and do not need to be pushed to Github. 

In [4]:

# Import libaries needed
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# made a change in colab

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

%cd gdrive/My Drive/G9A2/G9
 
#%cd gdrive/My Drive/G9A2
# I made a folder on my google drive that has the G9 Methylation data and the binfpy library
#this code automatically loads it in
# you can do the same with your google drive. 

Mounted at /content/gdrive
/content/gdrive/My Drive/G9A2/G9


In [ ]:
#set styles of plots - taken from Week 6 notebook
sns.set_theme(style="ticks")

sns.set(rc={'figure.figsize': (5, 5), 'font.family': 'sans-serif',
            'font.sans-serif': 'Arial', 'font.size': 10, 'axes.titlesize': 16,
           'axes.titleweight': 700}, style='ticks')
sns.set_palette("Set2")

# Read in the data


In [2]:
!ls

G9_liver_dna-meth.csv  G9_liver_gene-expr.csv


In [7]:
liver_meth_dna = pd.read_csv('G9_liver_dna-meth.csv')
liver_gene_expr = pd.read_csv('G9_liver_gene-expr.csv')


# TensorBoard for Model Evaluation

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# %reload_ext tensorboard

%tensorboard --logdir /dir/

# or use %tensorboard --logdir /dir/

# Solid Tissue Normal vs Primary Tumour Classifier


# Support Vector Machine

In [ ]:
## Encode the labels in 1 hot encoding
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

# Run a SVM
# https://scikit-learn.org/stable/modules/svm.html
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
plot_confusion_matrix(clf, X_test, y_test)  
plt.show()
plot_confusion_matrix(clf, data, labels) 

# Neural Network


In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=42)
# create model

model = Sequential()
model.add(Dense(2,  activation='sigmoid', input_shape=(3,),  name="bottleneck"))
model.add(Dense(3,  activation='softmax'))


#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs = 150, batch_size = 4)

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
encoder_p = Model(model.input, model.get_layer('bottleneck').output)

nn_enc = encoder_p.predict(data)  # bottleneck representation

# plot
for tissue in tissues:
    tissue_idxs = np.where(df['tissue'].values == tissue)
    sc = plt.scatter(nn_enc[tissue_idxs, 0], nn_enc[tissue_idxs, 1], s=60, label=tissue, marker='o', alpha=0.8)
    
ax.set_xlabel('AE 0')
ax.set_ylabel('AE 1')
plt.legend()
plt.title('AE of age, years smoked, BMI')
plt.show()


df['AE0'] = nn_enc[:, 0]
df['AE1'] = nn_enc[:, 1]

# plot each of the features on the plot! We'll relate these later 
cmaps = ['BuGn', 'PuRd', 'YlOrBr']
for i, s in enumerate(['bmi', 'smoke', 'age']):    
    sc = plt.scatter(df['AE0'], df['AE1'], s=60, label=s, c=df[s], marker='o', alpha=0.8, cmap=cmaps[i])

    ax.set_xlabel('BMI')
    ax.set_ylabel('Smoked years')
    plt.colorbar()
    plt.title(f'AE BMI, Smoked years and Age, colour: {s}')
    plt.show()

# Check Metrics of Models

In [ ]:
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical

predictions = model.predict(data)
prediction_ = np.argmax(predictions, axis = 1)
prediction_ = encoder.inverse_transform(prediction_)

for i, j in zip(prediction_ , labels):
    print( " the nn predict {}, and the tumour stage is {}".format(i,j))
confusion_matrix(prediction_, labels)


# New Section